
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# 4 - Building Dynamic Workloads with Advanced Tasks

In this demo, we will show how to build dynamic Lakeflow jobs using conditional logic (`if-else`) and iterative tasks (`for each` loop).

This demo will cover:
- Defining dependencies between tasks
- Adding a conditional `if-else` task
- Adding an iterative `for each` task

### Learning Objective
Create and visualize a dynamic Lakeflow job with multiple tasks and dependencies.

![Lesson04_final_job](./Includes/images/Lesson04_final_job.png)

After completing this demo, your job will look like above.

## REQUIRED - SELECT CLASSIC COMPUTE (The cluster named 'labuser')

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to **dbacademy** and the schema to your specific schema name shown below using the `USE` statements.
<br></br>


```
USE CATALOG dbacademy;
USE SCHEMA dbacademy.<your unique schema name>;
```

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

**NOTE:** If you use Serverless V1 a warning will be returned. You can ignore the warning.

In [0]:
%run ./Includes/Classroom-Setup-4

## B. Explore Your Schema
Complete the following to explore your **dbacademy.labuser** schema:

1. In the left navigation bar, select the catalog icon:  ![Catalog Icon](./Includes/images/catalog_icon.png)

2. Locate the catalog called **dbacademy** and expand the catalog.

3. Expand your **labuser** schema. 

4. Notice that within your schema you will find two tables named as **sales_bronze**, **customers_bronze** and **orders_bronze**.

**Note:** If you have completed the 2L Exercise, you may find additional tables under your schema.


## C. View Your Notebook

Follow these steps to view the notebook files used in this job. All files are located in the **Task Files** folder within the directory for the corresponding lesson number.

1. Navigate to (or click the link for) the notebook: [Task Files/Lesson 4 Files/4.1 - Joining Customers and Sales Table]($./Task Files/Lesson 4 Files/4.1 - Joining Customers and Sales Table)
   - Review the notebook. It creates a new table by joining the **customers_bronze** and **sales_bronze** tables. Pay attention to the code that sets the task value for the key **has_duplicates**.

2. Navigate to (or click the link for) the notebook: [Task Files/Lesson 4 Files/4.2 - Joining Customers and Orders Table]($./Task Files/Lesson 4 Files/4.2 - Joining Customers and Orders Table)
   - Review the notebook. It creates a new table by joining the **customers_bronze** and **orders_bronze** tables.

## D. Adding Task in Job
Complete the steps below to add new task into your Retail Job

###D1. Creating the Starter Job
1. Next, we will add the notebooks listed below as tasks to our job using the Databricks SDK. This approach avoids manually adding notebook tasks, as we've already done it in previous demonstrations and labs:


   - **4.1 - Joining Customers and Sales Table**  

   - **4.2 - Joining Customers and Orders Table**  

   Run the cell below to build the starter job that we have been continually building in this course. These commands will set up your job with all work completed so far and add the required tasks for this demonstration.

![Lesson04_starter_job](./Includes/images/Lesson04_starter_job.png)



In [0]:
job_tasks = [
        {
            'task_name': 'ingesting_orders',
            'file_path': '/Task Files/Lesson 1 Files/1.1 - Creating orders table',
            'depends_on': None
        },
        {
            'task_name': 'ingesting_sales',
            'file_path': '/Task Files/Lesson 1 Files/1.2 - Creating sales table',
            'depends_on': None
        },
        {
            'task_name': 'ingesting_customers',
            'file_path': '/Task Files/Lesson 3 Files/3.1 - Creating customers table',
            'depends_on': None
        }
        ,{
            'task_name': 'customers_sales_summary',
            'file_path': '/Task Files/Lesson 4 Files/4.1 - Joining Customers and Sales Table',
            'depends_on': [
                        {'task_key':'ingesting_customers'},
                        {'task_key': 'ingesting_sales'}
                        ]
        }
        ,{
            'task_name' : 'customers_orders_report',
            'file_path': '/Task Files/Lesson 4 Files/4.2 - Joining Customers and Orders Table',
            'depends_on': None
        }
    ]

myjob = DAJobConfig(job_name=f"Demo_04_Retail_Job_{DA.schema_name}",
                    job_tasks=job_tasks,
                    job_parameters=[
                        {'name':'catalog', 'default':'dbacademy'},
                        {'name':'schema', 'default':f'{DA.schema_name}'}
                    ])

### D2. Set Dependencies on the Tasks

In this step, we will modify the existing job to define task dependencies. Specifically, we'll configure the main task to run only after all preceding tasks have completed successfully.


Complete the following to review the job and set the following dependencies to the **customers_orders_report** task:
   - **ingesting_orders**
   - **ingesting_customers**

1. Navigate to **Jobs and Pipelines** and open it in a new tab.

2. Select your new job that starts with **Demo_04_Retail_Job_labuser**.

3. Click on **Tasks** in the top navigation bar.

4. Review your job. You should see five tasks: 
   - **customers_orders_report**.
   - **ingesting_customers**, 
   - **ingesting_orders**, 
   - **ingesting_sales**, 
   - **customers_sales_summary**,

5. Select the **customers_sales_summary** task. 
   - Notice that it depends on two tasks: **ingesting_customers** and **ingesting_sales**, with the dependency set to **All Succeeded**.

6. Next, select the **customers_orders_report** task and set the following task options: 

   - In the **Depends on**, add **ingesting_orders** and **ingesting_customers**

   - In **Run if dependencies**, set the dependency to **All Succeeded**.

   - Select **Save task**.

7. Click on **Run_now** to run the job.

<br></br>
#### Final Dependencies
![Lesson04_dependencies](./Includes/images/Lesson04_dependencies.png)


## E. Add an If/Else Conditional Task

In this section, you will add a conditional task to your job that checks for duplicate records in the **customers_sales_silver** table (Task **customers_sales_summary**). 

Based on the result, the workflow will branch to handle duplicates appropriately.

### E1. Checking for Duplication Logic

1. Recall the logic used to detect duplicates in the **customers_sales_silver** table. (The **customers_sales_summary** task creates the **customers_sales_silver** table.)

2. In that notebook, we check whether the **customers_sales_silver** table contains any duplicate records. If duplicates are found, the result of this check (a boolean value) is stored as `has_duplicates` in the task output.

**Code Reference:**

        df = spark.sql("""
            SELECT * FROM customers_sales_silver
        """)

        duplicate_exists = df.count() > df.dropDuplicates().count()

        dbutils.jobs.taskValues.set(key="has_duplicates", value=duplicate_exists)

**Notebook for Reference:**  
[Task Files/Lesson 4 Files/4.1 - Joining Customers and Sales Table]($./Task Files/Lesson 4 Files/4.1 - Joining Customers and Sales Table)

### E2. Create an If/Else Conditional Task

Create an **If/else conditional** task to determine what to execute based on whether duplicate records are found.

1. In your **Demo_04_Retail_Job_labuser** job, select **Add task**.

2. In the dialog box, scroll down to the **Advanced** section and select the task type **If/else condition**.

3. Name the new task **checking_for_duplicates**.

4. Set the **Depend on**  value to the **customers_sales_summary** task.

5. For the **Condition** field, use the parameter value created in the `customers_sales_summary` task:

   **Dynamic Value References:**
   This syntax leverages dynamic value references to access output variables from earlier tasks in your job. When a task runs (like `customers_sales_summary`), its results—including variables registered or output by the task (such as `has_duplicates`)—become available for downstream tasks.

   **By referencing** `tasks.customers_sales_summary.values.has_duplicates`, you dynamically pass the value (whether duplicates exist) to the If/Else condition. This enables conditional branching based on run-time data rather than static configuration, making your workflow adaptable and responsive to actual results.

    **Adding Condition Field:** 
     - To manually add the parameter value, select the `{}` in the **Condition** field. 
     - Find and click on `tasks.customers_sales_summary.values`, it will automatically add a suffix of `my_value` to it.
     - Replace `my_value` with the task parameter created in the notebook: `has_duplicates`.

6. Then set the condition to check if this value  `== true`

7. Select **Save task** to create the conditional task.


<br></br>
##### IF/ELSE CONDITION TASK

![Lesson04_conditional_task.png](./Includes/images/Lesson04_conditional_task.png)


### E3. Set the True Condition (Duplicates Exists) Task

Complete the following steps to add a task that runs **only if duplicates are found** (`tasks.customers_sales_summary.values.has_duplicates == true`).

1. Select the **checking_for_duplicates** task.

2. Click **Add task**, and choose **Notebook**.

3. Name the new task **dropping_duplicate_records**.

4. Use the notebook [4.3 - If Condition: Dropping Duplicates]($./Task Files/Lesson 4 Files/4.3 - If Condition: Dropping Duplicates) as the task source.  
   - This notebook includes logic to remove duplicate records from the **customers_sales_silver** table.

5. In the **Depends on** field, set this task to depend on the **True** branch of the **checking_for_duplicates** task (`checking_for_duplicates (true)`).

6. Click on **Create Task** 
<br></br>
##### TRUE DEPENDENCY TASK

![Lesson04_if_task](./Includes/images/Lesson04_if_task.png)

### E4. Set the False Condition (No Duplicates) Task

Complete the following steps to add a task that runs only if duplicates are not found (`tasks.customers_sales_summary.values.has_duplicates == false`).

This setup ensures your job automatically handles duplicates if they exist, or proceeds to data transformation if no duplicates are found.

1. Select the **checking_for_duplicates** task.

2. Click **Add task**, and choose **Notebook**.

3. Name the new task **transforming_customers_sales_table**.

4. Use the notebook [Task Files/Lesson 4 Files/4.4 - Else Condition: Cleaning and Transforming Customers Sales Table]($./Task Files/Lesson 4 Files/4.4 - Else Condition: Cleaning and Transforming Customers Sales Table) as the task source.  
   - This notebook includes logic to clean and transform the **customers_sales_silver** table.

5. In the **Depends on** field, set this task to depend on the following:
   - The **False** branch of the **checking_for_duplicates** task (`checking_for_duplicates (false)`).
   - The **dropping_duplicate_records** task.

6. In the **Run if dependencies** field, select **None Failed**.  
   - This ensures:
     - If there are no duplicates, the transformation runs immediately.
     - If duplicates exist, the job runs the task **dropping_duplicate_records**, then proceeds with the transformation task **transforming_customers_sales_table**.

7. Click on **Create Task**.

8. Click on **Run now** button to run the job


<br></br>
##### FALSE DEPENDENCY TASK

![Lesson04_false_task.png](./Includes/images/Lesson04_false_task.png)

### E5. Job Confirmation  
Confirm your job looks like the following after adding the **If/else condition** and associated tasks:


![Lesson04_IfElse](./Includes/images/Lesson04_IfElse.png)

## F. Add a For Each Loop Task

In this section, you will add a downstream task to **customers_orders_report** that uses a **For Each** loop. This loop allows the job to execute the same task multiple times, once for each item in a specified list or collection. Execution may happen sequentially or concurrently, depending on job configuration. 

In our case, from the customers_orders_silver table, we want to generate orders reports specifically for the states of **California, New York, and Virginia**. We will create three different tables to store state-specific data. We will use the same code script and dynamically pass the state name with the help of the **For Each** task.

### F1. Explore the Notebooks

1. Review the notebook [Task Files/Lesson 4 Files/4.2 - Joining Customers and Orders Table]($./Task Files/Lesson 4 Files/4.2 - Joining Customers and Orders Table), which creates the **customers_orders_silver** table.

2. The [Task Files/Lesson 4 Files/4.5 - For Each: Customer orders State]($./Task Files/Lesson 4 Files/4.5 - For Each: Customer orders State) notebook will be executed in a loop for each state mentioned above. This script dynamically takes the state value and runs it for each state, creating a state-specific table with customers_order_silver data.

### F2. Creating a For Each Iterator Task (Part 1 of 2)

The "For Each" task involves two steps: first, define the iterator, and then specify the script to be iterated. Now, complete the following to add a **For Each** iterator task to loop over a series of **state** values.

1. In the same job, select the **customers_orders_report** task.

2. Select **Add task** and select the task type **For each**.

3. Name the task **customers_orders_state_wise_report_iterator**. 

4. Set the iterator **Inputs** field to `["CA", "NY", "VA"]`.  
  — These are the states with the highest number of customers.

5. Leave the **Concurrency** setting blank (recommended for single-node runs to avoid slowing down the process).

6. Set the **Depends on** field for this task to **customers_orders_report**.

7. Ensure the **Run if dependencies** is set to **All succeeded**.

8. Click on **Add a task to loop over**.

#### For Each Iterator

![Lesson04_For_Each_Task_Iterator.png](./Includes/images/Lesson04_For_Each_Task_Iterator.png)

### F3. Add a Task to Loop Over (Part 2 of 2)

Now that the **For Each** task iterator is set, we need to specify a task to loop over. Complete the following to add the task to loop over.

1. Now that the iterator is set. Select **Add a task to loop over**. 

2. Name the task to iterate over **customers_orders_state_wise_report**.

3. Confirm the task **Type** is **Notebook** and the **Source** is **Workspace**.

4. Set the notebook path to [Task Files/Lesson 4/4.5 - For Each: Customer orders State]($./Task Files/Lesson 4 Files/4.5 - For Each: Customer orders State), which is under the **Task Files** folder.

5. Set **Compute** to **serverless**.

6. Add a key-value parameter:
   - For key, add **state**. 
   - For the value, click on the **{}** symbol and select **input**.  
   - This will automatically pass each state code from the iterator loop to the notebook.

7. Click **Create task**.

<br></br>
#### Iterator Task
![Lesson04_iterator_task.pngg](./Includes/images/Lesson04_iterator_task.png)

## G. Run the Entire Job

To execute your job:

1. Click on **Run Now** to start the job.

2. Go to the **Runs** tab to monitor the progress and view the results of each task.

This will run all tasks in your job according to the dependencies and logic you have set up.

**NOTE:** This job will take about 5 minutes to complete.

## H. Conclusion and Results

When your job run is successful, Click on catalog icon, go to your schema under dbacademy catalog. Look out for new tables **customers_sales_gold** , **customers_orders_ca_silver**, **customers_orders_ny_silver** and **customers_orders_va_silver**.

The `customers_sales_gold` table does not require any transformation. It is our gold-tier table containing sales metrics such as **units_purchased, avg_price_per_unit, total_price**, customer details like **customer_id, customer_name, loyalty_segment**, and supporting order details.

In [0]:
%sql
SELECT * 
FROM customers_sales_gold

The tables **customers_orders_ca_silver**, **customers_orders_ny_silver**, and **customers_orders_va_silver** are state-specific and contain relevant data for each state. These tables will be further transformed to add business columns, which we will do in a future demo to create gold-tier tables. Now, query them to see the type of data they contain.

In [0]:
%sql
SELECT * 
FROM customers_orders_ca_silver

In [0]:
%sql
SELECT * 
FROM customers_orders_ny_silver

In [0]:
%sql
SELECT * 
FROM customers_orders_va_silver

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>